A rotatividade de clientes é um problema crucial no setor de telecomunicações, definido como a perda de clientes para outra operadora. Prever a rotatividade antecipadamente, como "este cliente vai nos deixar nos próximos X meses?", permite aplicar políticas de marketing para reter clientes e aumentar a base. Mesmo uma redução de 1% na taxa de rotatividade pode resultar em um aumento significativo nos lucros.

A análise de churn envolve estudar "um cliente", "um produto ou serviço" e "a probabilidade de abandono". O objetivo é identificar clientes propensos a sair e tomar ações preventivas antes que eles partam. 

Os dados de rotatividade que utilizo são de uma empresa fictícia que forneceu serviços de telefone e Internet para 7.043 clientes na Califórnia no terceiro trimestre. Com esses dados, desenvolvo um modelo de aprendizado de máquina para prever quais clientes deixarão a empresa, realizando análise de dados e engenharia de recursos.

O conjunto de dados Telco Customer Churn da Kaggle foi utilizado, contendo 21 colunas e 7.043 linhas com informações como customerID, sexo, serviço telefônico e de Internet. Analiso essas colunas para identificar variáveis independentes (X) e dependentes.

Esse projeto me proporciona experiência prática em ciência de dados, incluindo a manipulação de grandes volumes de dados, aplicação de técnicas de aprendizado de máquina, e implementação de estratégias de retenção de clientes, preparando-me para atuar em diversos setores e segmentos de negócios.


### 0.0 - Imports

In [6]:
# mmanipulação
import pandas as pd
import inflection
import gc

# matematica e estatistica
import numpy as np
from scipy import stats

# visualização
import matplotlib.pyplot as plt
import seaborn as sns

gc.collect()

0

### 0.1 - Help Functions

480

### 0.2 - Path Definition

In [16]:
HOME_PATH = 'C:/Users/valmi/projeto_churn/'
RAW_DATA_PATH = '/data/raw/'
INTERIM_DATA_PATH = '/data/interim/'
FIGURE_PATH = '/reports/figures/'

### 0.3- Load Data

In [17]:
base_churn_raw = pd.read_csv(HOME_PATH+RAW_DATA_PATH+'WA_Fn-UseC_-Telco-Customer-Churn.csv')
base_churn_raw.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
